In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train/weight.pth
/kaggle/input/train/__results__.html
/kaggle/input/train/submission.csv
/kaggle/input/train/__notebook__.ipynb
/kaggle/input/train/__output__.json
/kaggle/input/train/custom.css
/kaggle/input/train/__results___files/__results___10_0.png
/kaggle/input/train/__results___files/__results___12_1.png
/kaggle/input/train/__results___files/__results___11_1.png
/kaggle/input/nycu-ml-pattern-recognition-hw-4/released/test/7c08c8c0-0064-40c3-a7cd-dda2b4043347.pkl
/kaggle/input/nycu-ml-pattern-recognition-hw-4/released/test/cb206033-5968-44f9-bfc8-dab33bb57bae.pkl
/kaggle/input/nycu-ml-pattern-recognition-hw-4/released/test/355823c5-bc8e-4352-8892-10bf14ce866a.pkl
/kaggle/input/nycu-ml-pattern-recognition-hw-4/released/test/7f6d455e-47a0-4529-954d-0cc432f63d50.pkl
/kaggle/input/nycu-ml-pattern-recognition-hw-4/released/test/7a2304f3-f864-4c16-b683-468d87808661.pkl
/kaggle/input/nycu-ml-pattern-recognition-hw-4/released/test/fffd07c4-9764-423a-9e8d-10f1a3519c5d.pkl
/k

In [2]:
import pickle
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# GPU device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [3]:
# get file name
testFileName = list()
trainFileNameClass0 = list()
trainFileNameClass1 = list()
os.chdir('/kaggle/input/nycu-ml-pattern-recognition-hw-4/released/test')
testFileName = os.listdir()
os.chdir('../train/class_0')
trainFileNameClass0 = os.listdir()
os.chdir('../class_1')
trainFileNameClass1 = os.listdir()
os.chdir('../../')
# test file name
testFileName[0], trainFileNameClass0[0], trainFileNameClass1[0]

('7c08c8c0-0064-40c3-a7cd-dda2b4043347.pkl',
 '32767294-e76e-485e-91a0-c2a123498e01.pkl',
 '3cc6a03d-7585-48f4-82cc-89448620b059.pkl')

In [4]:
def readData(filepath, filenamelist):
    FileExtensionRemove = []
    with open(filepath+filenamelist[0], 'rb') as f:
        data = np.array([pickle.load(f)])
    FileExtensionRemove.append(filenamelist[0][:-4])
    for filename in filenamelist[1:]:
        with open(filepath+filename, 'rb') as f:
            data = np.concatenate((data, np.array([pickle.load(f)])), axis=0)
        FileExtensionRemove.append(filename[:-4])
    return data, FileExtensionRemove

def DataPreprocess(trainX0, trainX1=None):
    # combine train data
    if trainX1 is None:
        trainX = trainX0
    else:
        trainX = np.concatenate((trainX0, trainX1))
    # Transform to 0 ~ 1
    trainX = trainX.astype('float32')
    trainX /= 255
    # To pytorch
    trainX = torch.from_numpy(trainX)
    if trainX1 is None:
        trainY = None
    else:
        trainY = torch.concatenate((torch.zeros((trainX0.shape[0],1)), 
                                    torch.ones((trainX1.shape[0],1))))
    return trainX, trainY

In [5]:
# Feature extraction model using AlexNet
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        base_model = models.alexnet(weights='IMAGENET1K_V1')
        self.feature_extractor = nn.Sequential(
            *list(base_model.features), 
            # nn.Dropout(p=0.2),
            nn.AdaptiveAvgPool2d((6, 6)),
            nn.Flatten(),
            *list(base_model.classifier[:-1])
        )
    
    def forward(self, x):
        x = self.feature_extractor(x)
        return x

In [6]:
# Number of tiles per bag
FEATURE_DIM = 4096
# MIL model
class MILModel(nn.Module):
    def __init__(self):
        super(MILModel, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.classifier = nn.Linear(FEATURE_DIM, 1)
    
    def forward(self, x):
        batch_size = x.size(0)
        num_tiles = x.size(1)
        x = x.view(-1, 3, 128, 128)
        features = self.feature_extractor(x)
        features = features.view(batch_size, num_tiles, -1)
        aggregated_features, _ = torch.max(features, dim=1)
        logits = self.classifier(aggregated_features)
        output = torch.sigmoid(logits)
        
        return output
    
    def predict(self, x):
        with torch.no_grad():
            ypred_prob = self(x)
            ypred = ypred_prob.clone().detach()
            ypred[ypred < 0.5] = 0
            ypred[ypred >= 0.5] = 1
            return ypred_prob, ypred

In [7]:
# Set random seed for reproducibility
torch.manual_seed(0)

# Initialize the model
model = MILModel().to(device)
model.load_state_dict(torch.load("/kaggle/input/train/alexnet-weight.pth"))

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 167MB/s]  


<All keys matched successfully>

In [9]:
model.eval()

# load test data and predict

predictArray = np.array([])
DataSizeBatch = 25
FileExtensionRemove = []
for i in range(int(len(testFileName)/DataSizeBatch)):
    print("Predict Batch ", i)
    
    startIndex = i * DataSizeBatch
    testX, Name = readData('test/', testFileName[startIndex:startIndex + DataSizeBatch])
    FileExtensionRemove += Name
    testX, _ = DataPreprocess(testX)
    
    pred_prob, pred_label = model.predict(testX.to(device))
    predictArray = np.concatenate((predictArray, pred_label.cpu().detach().numpy().T[0]))

predictArray = predictArray.astype(np.int64)

Predict Batch  0
Predict Batch  1
Predict Batch  2
Predict Batch  3
Predict Batch  4
Predict Batch  5
Predict Batch  6
Predict Batch  7


In [10]:
# to pandas
submission = pd.DataFrame()
submission["image_id"] = FileExtensionRemove
submission["y_pred"] = predictArray

In [11]:
submission

,image_id,y_pred
0,7c08c8c0-0064-40c3-a7cd-dda2b4043347,1
1,cb206033-5968-44f9-bfc8-dab33bb57bae,1
2,355823c5-bc8e-4352-8892-10bf14ce866a,1
3,7f6d455e-47a0-4529-954d-0cc432f63d50,1
4,7a2304f3-f864-4c16-b683-468d87808661,1
...,...,...
195,4e4aef19-6408-46d9-9573-f8948c640e9d,1
196,9ea39c64-6d6c-4ff6-ae08-71ebca779275,1
197,e072b562-f2ef-42f7-8c60-ecf744a9469e,1
198,80b8987c-70c4-4274-9b46-893e3877a8cd,1


In [12]:
submission.to_csv('/kaggle/working/submission.csv', index = None)